In [1]:
import pandas as ps
import os
import numpy as np 
from transformers import BertTokenizer, TFBertForMaskedLM
import tensorflow as tf
import transformers
import glob

2023-10-23 17:43:39.683471: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-23 17:43:39.900483: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-23 17:43:40.381544: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/wsl/lib::/home/larkinan/miniconda3/envs/tf/lib/:/home/larkinan/min

In [2]:
def mapTextExtract(tweet):
    tmpText = tweet['text'].replace("\n",". ")
    
    return(tmpText + "\n")


In [3]:
def mapConvExtract(tweet):
    try:
        convid = tweet['conversation_id']
    except:
        convid = 'nope'
    return(convid)

In [4]:
def mapIdExtract(tweet):
    return(tweet['id'])

In [5]:
def processSingleYearSingleVariable(tuple):
    year = tuple[0]
    variable = tuple[1]
    folder = "Z:/Aspire/TweetStore/" + str(year) + "/" + str(variable) + "/"
    outputFile = "H:/Aspire/BERT/BERT_text/txt_" + str(year) + "_" + str(variable) + ".txt"
    if(os.path.exists(outputFile)):
        print("%s already exists" %(outputFile))
        return
    tweetFiles = glob.glob(folder + "tw_*")
    text = []
    print("found %i files for variable %s and year %i" %(len(tweetFiles),variable,year))
    for file in tweetFiles:
        text += combineSingleDay(file)
    writeToText(text,outputFile)

In [6]:
def loadSingleBatch(filename):
    tweetBatch = np.load(filename,allow_pickle=True)
    text,convId,tweetId = [],[],[]
    for subBatch in tweetBatch:
        text +=  list(map(mapTextExtract,subBatch))
        convId += list(map(mapConvExtract,subBatch))
        tweetId += list(map(mapIdExtract,subBatch))
    df = ps.DataFrame({
        'text':text,
        'convId':convId,
        'tweetId':tweetId
    })
    if(df.count()[0]>0):
        df = df[~df['convId'].str.contains('nope')]
    return(df)

In [7]:
def setupModelInputs(tweetData,debug=False):
    tweetText = list(tweetData['text'])
 #   tweetText = list(map(mapNewLineReplace,tweetText))
    inputs = TOKEN(tweetText,max_length=100,truncation=True,padding='max_length',return_tensors="tf")
    inp_ids = tf.convert_to_tensor(inputs['input_ids'])
    inputs['input_ids'] = inp_ids
    return(inputs)

In [8]:
def combineSingleDay(inFile):
    text = []
    npData = np.load(inFile,allow_pickle=True)
    for batch in npData:
        text += list(map(mapTextExtract,batch))
    print("found %i tweets for day %s" %(len(text),inFile))
    return(text)

In [9]:
def processSingleYearSingleVariable(year,variable,model):
    inFolder = "/mnt/z/Aspire/TweetStore/" + str(year) + "/" + str(variable) + "/"
    outFolder = "/mnt/h/Aspire/PredStore/" + str(year) + "/" + str(variable) + "/"
    tweetFiles = glob.glob(inFolder + "tw_*")
    for file in tweetFiles:
        dateStamp = file[file.rfind('_')+1:-4]
        outputFile = outFolder + "re_" + dateStamp + ".csv"
        if(os.path.exists(outputFile)):
            print("%s already exists" %(outputFile))
        else:
            processSingleBatch(file,outputFile,model)

In [10]:
modelFile = '/mnt/h/Aspire/BERT/place/place_model_checkpoint.h5'
relaxedBERT = tf.keras.models.load_model(modelFile,custom_objects={"TFBertModel": transformers.TFBertModel})
relaxedBERT.compile(loss=[tf.keras.losses.BinaryCrossentropy()],optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics=['accuracy'])
TOKEN = BertTokenizer.from_pretrained('/mnt/h/Aspire/BERT/place/expandedTokenBase')

2023-10-23 17:43:42.709777: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-23 17:43:42.714417: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-23 17:43:42.714449: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-23 17:43:42.728561: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-23 17:43:42.728619: I tensorflow/compile

In [11]:
def processSingleBatch(inFile,outFile,model):  
    df = loadSingleBatch(inFile)
    if(df.count()[0]==0):
        #print("%s has 0 records" %(inFile))
        return
    modelInputs = setupModelInputs(df)
    preds = model.predict([modelInputs['input_ids'],modelInputs['attention_mask']])
    df['isDaycare'] = tf.nn.sigmoid(preds[:,0])
    df['isPark'] = tf.nn.sigmoid(preds[:,1])
    df['isHome'] = tf.nn.sigmoid(preds[:,2])
    df['isSchool'] = tf.nn.sigmoid(preds[:,3])
    df['isNeighborhood'] = tf.nn.sigmoid(preds[:,4])
    df['isIndoor'] = tf.nn.sigmoid(preds[:,5])
    df['isOutdoor'] = tf.nn.sigmoid(preds[:,6])
    df2 = df.drop(['text'],axis=1)
    df2.to_csv(outFile,index=False)

In [12]:
a = np.load("/mnt/z/Aspire/TweetStore/2016/place/tw_20160204.npy",allow_pickle=True)
index=0
for b in a:
    for c in b:
        try:
            d = c['conversation_id']
        except:
            print(index)
        index+=1

20281
20307
20336
20464
20521
20587
20594
20653
20674
20693


In [13]:
def processSingleConvFolder(folder,model):
    inFolder = "/mnt/z/Aspire/TweetStore/conversations/" + folder + "/"
    outFolder = "/mnt/h/Aspire/PredStore/conversations/" + folder + "/"
    tweetFiles = glob.glob(inFolder + "tw_*")
    for file in tweetFiles:
        convId = file[file.rfind('_')+1:-4]
        outputFile = outFolder + "re_" + convId + ".csv"
        if(os.path.exists(outputFile)): 
            a=1
         #   print("%s already exists" %(outputFile))
        else:
            processSingleBatch(file,outputFile,model)

In [14]:
parentConvFolder = '/mnt/z/Aspire/TweetStore/conversations/'
outputFolder = '/mnt/h/Aspire/PredStore/conversations/'
conversationFolders = os.listdir(parentConvFolder)
index = 0
for folder in conversationFolders:
    curFolder = parentConvFolder + folder + "/"
    if not(os.path.exists(outputFolder + folder)):
        os.mkdir(outputFolder + folder)
    processSingleConvFolder(folder,relaxedBERT)
    index+=1
    if(index%100==0):
        print(index)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
  1/958 [..............................] - ETA: 30:33

2023-10-23 18:06:03.875281: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


26/26 [==============================] - 1s 35ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 33ms/step


...

2/2 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 37ms/step


2/2 [==============================] - 0s 28ms/step


 3/12 [======>.......................] - ETA: 0s

In [ ]:
for year in reversed(range(2014,2015)):
    yearFolder = '/mnt/h/Aspire/PredStore/' + str(year) + "/"
    if not(os.path.exists(yearFolder)):
        os.mkdir(yearFolder)
    for cat in['age','place','env','health','health2']:
        print("calculating values for year %i and category %s" %(year,cat))
        catFolder = yearFolder + "/" + str(cat)
        if not(os.path.exists(catFolder)):
            os.mkdir(catFolder)
        processSingleYearSingleVariable(year,cat,relaxedBERT)